<a href="https://colab.research.google.com/github/gened1080/visualizing-sound/blob/master/Visualizing_Sound_Fall_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Visualize Sound (no audio/video hardware required)

This notebook helps you visualize the propagation of sound and its interaction with boundaries. To create the boundaries you can use an image. An edge detection algorithm is appied on the frame/image to generate the boundaries. A sound source is added to the center of the frame. The sound source can specified by providing a numerical frequency. The propagation medium can also be customized by entering sound velocity, medium density. In addition, a block can be inserted in the right half of the frame that is a different temperature (which can be entered) than the rest of the medium. 

The notebook relies on functions defined in the python file `VizSound_noAV.py` that we import in the following cell. If you are interested to see how the functions are implemented then open the `VizSound_noAV.py` file in another window to take a look.

In [8]:
%%bash
!(stat -t /usr/local/lib/*/dist-packages/google/colab > /dev/null 2>&1) && exit 
rm -rf visualizing-sound
git clone https://github.com/gened1080/visualizing-sound.git
pip install pydub
pip install datasketch
sudo apt-get install libasound-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg
pip install pyaudio

Reading package lists...
Building dependency tree...
Reading state information...
libportaudio2 is already the newest version (19.6.0-1).
libportaudiocpp0 is already the newest version (19.6.0-1).
portaudio19-dev is already the newest version (19.6.0-1).
libasound2-dev is already the newest version (1.1.3-5ubuntu0.5).
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


Cloning into 'visualizing-sound'...


In [9]:
# Import relevant libraries
import matplotlib.pyplot as plt
from matplotlib import rc
rc('animation', html='jshtml')
import matplotlib.animation as animation
from time import time
import sys
sys.path.append('/content/visualizing-sound/')
import VizSound_noAV as vs

### Setting up the simulation

In order to setup the simulation, we have to go through two main steps. The first step is provide the frame (`.jpg` image file) from which boundaries are created. The code will then perform some image processing operations on the frame, such as resizing, edge detection, noise reduction, and thresholding to generate the boundaries. The second step is to choose the properties of the sound source and the propagating medium (we call this the sound set up). There is a default sound setup that you can use or you can customize everything. For setting up the simulation, simply run the cell below and follow the instructions.

In [10]:
# Creating an object of the VSfdtd class defined in VizSound.py
vs_obj = vs.VSfdtd()

Enter the filename of your image including extension: exp.png

Resized the frame to  self.c  pixels wide

************************************
Step 2. Determine the sound setup, i.e., the sound source and the propagation medium
The default sound setup is the following:
Single frequency sinusoidal point source placed at the center of the window
frequency of sound source = 15000 Hz
medium = air
sound velocity = 346.13 m/s
density of medium = 1.2 kg/m^3

You can also customize everything by entering "c" below and following instructions

Enter "d" to run the default sound setup or "c" to customize: c
************************************


KeyboardInterrupt: ignored

### Running the simulation

This code simulates sound propagation by solving the [acoustic wave equation](https://en.wikipedia.org/wiki/Acoustic_wave_equation#Equation) using a method known as the [finite-difference time-domain](https://en.wikipedia.org/wiki/Finite-difference_time-domain_method) (FDTD). The FDTD method was originally estalished for solving the electromagnetic wave equation, however one can make a direct correspondence between the electromagnetic and acoustic wave equations and apply the same method. In brief, the FDTD method converts the coupled partial differential equations into algebraic equations that are solved in time and space. Running the simulation essentially means running the loop in which these algebraic equations are solved. The results (sound pressure) are displayed in the animation below the cell. To run the simulation all you have to do is run the cells below.

In [ ]:
# Setup a figure 
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111)

In [ ]:
# This function runs the FDTD simulation and generates the frames to create the animation
def propagate_sound(i):
    global vs_obj, ax, fig
    vs_obj.fdtd_update()
    vs_obj.source(i)
    if vs.bflag == "o":
        vs_obj.boundary()
    imgdisp = vs_obj.img_cap + vs_obj.pr + vs_obj.mbndry
    ax.clear()
    imsound = ax.pcolormesh(imgdisp, cmap="gray", vmin=-1, vmax=1)
    return imsound

In [ ]:
# choosing an appropriate interval between frames based on dt and 
# the time to animate one step
t0 = time()
propagate_sound(0)
t1 = time()
interval = 1000 * vs_obj.dt - (t1 - t0)
# setup animatation
vis_sound = animation.FuncAnimation(fig, propagate_sound, frames=300, interval=interval, blit=False, repeat=True)

In [ ]:
# Create and run the animation
vis_sound